![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
![table example](img/columns.png)

### Terminology

- Schema
- Primary Key
- Foreign Key
- Structured queries
- Views

***
### SQLite

![sqlite](img/SQLite-Python.jpg)

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to an Oracle database server there is a different package, `cx_Oracle`. The documentation for that package is [here](https://oracle.github.io/python-cx_Oracle/)

***
## Goal 3: Get going with sqlite!

In [99]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [52]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [53]:
cursor.execute("SELECT name,type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [54]:
cursor.execute("SELECT * FROM airports as a").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

#### Use description

In [27]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [28]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [38]:
cursor.execute("SELECT * FROM airlines as a").fetchone()

(0, '1', 'Private flight', '\\N', '-', None, None, None, 'Y')

In [39]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('alias', None, None, None, None, None, None),
 ('iata', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('callsign', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('active', None, None, None, None, None, None))

In [40]:
cursor.execute("SELECT * FROM routes").fetchone()

(0, '2B', '410', 'AER', '2965', 'KZN', '2990', None, '0', 'CR2')

In [41]:
cursor.description

(('index', None, None, None, None, None, None),
 ('airline', None, None, None, None, None, None),
 ('airline_id', None, None, None, None, None, None),
 ('source', None, None, None, None, None, None),
 ('source_id', None, None, None, None, None, None),
 ('dest', None, None, None, None, None, None),
 ('dest_id', None, None, None, None, None, None),
 ('codeshare', None, None, None, None, None, None),
 ('stops', None, None, None, None, None, None),
 ('equipment', None, None, None, None, None, None))

In [42]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

In [98]:
cursor.execute("""
SELECT name
FROM airlines 
WHERE active = 'Y'
AND country = 'United Kingdom'
""").fetchmany(10)

AttributeError: 'builtin_function_or_method' object has no attribute 'execute'

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [79]:
cursor.execute("""SELECT r.dest FROM routes as r""")

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

![venn](img/venn.png)

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [82]:
cursor.execute("""
SELECT a.latitude, a.longitude, r.airline, r.source, r.dest
FROM routes as r
JOIN airports as a on r.source_id =a.id
""").fetchone()

('43.449928', '39.956589', '2B', 'AER', 'KZN')

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airports?
- Which countries have the highest amount of inactive airports?
- What about airports by timezones?

In [87]:
cursor.execute("""
SELECT country, count(*) AS count
FROM airlines
WHERE active = 'Y'
GROUP BY country
ORDER BY 2 DESC
LIMIT 10
""").fetchall()


[('United States', 141),
 ('Russia', 72),
 ('United Kingdom', 40),
 ('Germany', 37),
 ('Canada', 34),
 ('Australia', 26),
 ('China', 25),
 ('Spain', 24),
 ('Brazil', 23),
 ('France', 22)]

In [91]:
cursor.close()
con.close()

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [92]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [93]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()
#vs
shelter_data[shelter_data['Name']=="Max"]

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
126,A800791,Max,07/29/2019 06:07:00 PM,07/29/2019 06:07:00 PM,07/26/2017,Return to Owner,NaN,Dog,Intact Male,2 years,Pit Bull,Brown/White
262,A788916,Max,07/27/2019 05:16:00 PM,07/27/2019 05:16:00 PM,11/26/2018,Return to Owner,NaN,Dog,Neutered Male,7 months,Pit Bull Mix,Brown Brindle/White
290,A799870,Max,07/26/2019 07:17:00 PM,07/26/2019 07:17:00 PM,07/13/2017,Return to Owner,NaN,Dog,Intact Male,2 years,German Shepherd,Brown/Black
1170,A799881,Max,07/13/2019 02:33:00 PM,07/13/2019 02:33:00 PM,05/13/2019,Transfer,Partner,Dog,Intact Male,1 month,Yorkshire Terrier Mix,Brown/White
1535,A798950,Max,07/07/2019 01:31:00 PM,07/07/2019 01:31:00 PM,07/02/2017,Adoption,NaN,Dog,Neutered Male,2 years,American Bulldog,White/Blue
1684,A799025,Max,07/05/2019 11:13:00 AM,07/05/2019 11:13:00 AM,01/19/2019,Return to Owner,NaN,Dog,Neutered Male,5 months,German Shepherd Mix,Tan
1964,A798743,Max,06/30/2019 04:54:00 PM,06/30/2019 04:54:00 PM,10/29/2018,Return to Owner,NaN,Dog,Intact Male,8 months,Maltese/Miniature Poodle,White/Tan
2419,A700767,Max,06/25/2019 02:12:00 PM,06/25/2019 02:12:00 PM,04/19/2011,Return to Owner,NaN,Dog,Neutered Male,8 years,Labrador Retriever/Plott Hound,Brown Brindle/White
2729,A797355,Max,06/21/2019 06:39:00 PM,06/21/2019 06:39:00 PM,12/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Pit Bull Mix,Chocolate/White
2741,A797407,Max,06/21/2019 04:05:00 PM,06/21/2019 04:05:00 PM,03/13/2019,Adoption,NaN,Dog,Neutered Male,3 months,German Shepherd/Chinese Sharpei,Tricolor


In [ ]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

## Goal 6: Transfering from sqlite to pandas

In [ ]:
conn = sqlite3.connect("flights.db")
df = pd.read_sql_query(";", conn)
df

**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [4]:
import sqlite3
conn = sqlite3.connect("Chinook_Sqlite.sqlite")
cursor = conn.cursor()

In [6]:
cursor.execute("SELECT name,type FROM sqlite_master")
len(cursor.fetchall())

33

In [9]:
cursor.execute("SELECT name,type FROM sqlite_master")
print(cursor.fetchall())

[('Album', 'table'), ('Artist', 'table'), ('Customer', 'table'), ('Employee', 'table'), ('Genre', 'table'), ('Invoice', 'table'), ('InvoiceLine', 'table'), ('MediaType', 'table'), ('Playlist', 'table'), ('PlaylistTrack', 'table'), ('sqlite_autoindex_PlaylistTrack_1', 'index'), ('Track', 'table'), ('IPK_Album', 'index'), ('IPK_Artist', 'index'), ('IPK_Customer', 'index'), ('IPK_Employee', 'index'), ('IPK_Genre', 'index'), ('IPK_Invoice', 'index'), ('IPK_InvoiceLine', 'index'), ('IPK_MediaType', 'index'), ('IPK_Playlist', 'index'), ('IPK_PlaylistTrack', 'index'), ('IPK_Track', 'index'), ('IFK_AlbumArtistId', 'index'), ('IFK_CustomerSupportRepId', 'index'), ('IFK_EmployeeReportsTo', 'index'), ('IFK_InvoiceCustomerId', 'index'), ('IFK_InvoiceLineInvoiceId', 'index'), ('IFK_InvoiceLineTrackId', 'index'), ('IFK_PlaylistTrackTrackId', 'index'), ('IFK_TrackAlbumId', 'index'), ('IFK_TrackGenreId', 'index'), ('IFK_TrackMediaTypeId', 'index')]


In [26]:
cursor.execute("PRAGMA table_info(InvoiceLine)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'InvoiceLineId', 'INTEGER', 1, None, 1)
(1, 'InvoiceId', 'INTEGER', 1, None, 0)
(2, 'TrackId', 'INTEGER', 1, None, 0)
(3, 'UnitPrice', 'NUMERIC(10,2)', 1, None, 0)
(4, 'Quantity', 'INTEGER', 1, None, 0)


## Reflection